### Model Results

###### prefix:
monetary = avg_customer loan amount
recency = days since customer last loan
frequency = number of loans customer has taken

##### format: 
[Model used] (parameter - if any) {f1-score, precision, recall}
feature importance(top10):

- starter logistic regressor {0.26, 0.16, 0.8}
lender_portion_to_be_repaid, amount_funded, loan_type, amount_to_repay, tbl_loan

- base lr {0.44, 0.28, 0.96}
loan type, pct_repay, lender id, lender portion to repay, loan type

- dec tree {0.83, 0.83, 0.83}
pct repay, lender portion to be repaid, inflation, loan type, monetary, dis_day_of_year

- rf {0.85, 0.94, 0.78}
pct repay, lender portion, inflation, loan type, monetary, dis_day_of_year

- xgb {0.87, 0.92, 0.83}
loan type, pct repay, unemployment rate, dis_year

lgb {0.85, 0.79, 0.92}
pct repay, frequency, dis_day_of_year, recency, duration, lender portion funded

knn {0.70, 0.56, 0.92}

voting_clf (est={rf:2, lgb:1, lr:1}, voting=soft) {0.88, 0.88, 0.89}
lender portion to be repaid, monetary, recency, dis_day_of_year, pct repay, dis_day_of_month

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_data = pd.read_csv('train_processed.csv')
test_data = pd.read_csv('test_processed.csv')

In [4]:
df = pd.concat([train_data, test_data])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87248 entries, 0 to 18593
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           87248 non-null  object 
 1   customer_id                  87248 non-null  int64  
 2   country_id                   87248 non-null  object 
 3   tbl_loan_id                  87248 non-null  int64  
 4   lender_id                    87248 non-null  object 
 5   loan_type                    87248 non-null  object 
 6   total_amount                 87248 non-null  float64
 7   total_amount_to_repay        87248 non-null  float64
 8   disbursement_date            87248 non-null  object 
 9   due_date                     87248 non-null  object 
 10  duration                     87248 non-null  int64  
 11  new_versus_repeat            87248 non-null  int64  
 12  amount_funded_by_lender      87248 non-null  float64
 13  lender_portion_funded

In [24]:
df.describe(include='all')

,id,customer_id,country_id,tbl_loan_id,lender_id,loan_type,total_amount,total_amount_to_repay,disbursement_date,due_date,...,due_month,due_day,due_day_of_week,due_day_of_year,binned_lender_portion,binned_duration,pct_repay,monetary,recency,frequency
count,87248,87248.000000,87248,87248.000000,87248,87248,8.724800e+04,8.724800e+04,87248,87248,...,87248.000000,87248.000000,87248.000000,87248.000000,87248,87248,87248.000000,8.724800e+04,87248,87248.000000
unique,87248,NaN,2,NaN,8,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27,14,NaN,NaN,NaN,NaN
top,ID_257604246395267278,NaN,Kenya,NaN,lender_A,B,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"(0.273, 0.311]","(-0.095, 37.5]",NaN,NaN,NaN,NaN
freq,1,NaN,83723,NaN,78874,75341,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,51559,85805,NaN,NaN,NaN,NaN
mean,NaN,256116.341761,NaN,267182.282539,NaN,NaN,1.475760e+04,1.567067e+04,2022-10-15 10:11:16.031542528,2022-10-25 00:45:40.106363648,...,8.950486,15.967266,2.557147,257.150697,NaN,NaN,1.023385,4.116875e+05,694 days 05:33:13.764900056,31.703214
min,NaN,145.000000,NaN,101323.000000,NaN,NaN,2.000000e+00,0.000000e+00,2021-10-04 00:00:00,2021-10-21 00:00:00,...,1.000000,1.000000,0.000000,1.000000,NaN,NaN,0.000000,3.000000e+01,0 days 00:00:00,1.000000
25%,NaN,249194.000000,NaN,235931.750000,NaN,NaN,2.259000e+03,2.291000e+03,2022-08-08 00:00:00,2022-08-17 00:00:00,...,8.000000,8.000000,1.000000,222.000000,NaN,NaN,1.000000,5.632700e+04,717 days 00:00:00,13.000000
50%,NaN,256357.000000,NaN,262505.500000,NaN,NaN,5.099000e+03,5.218000e+03,2022-09-16 00:00:00,2022-09-23 00:00:00,...,9.000000,17.000000,3.000000,260.000000,NaN,NaN,1.010834,1.467720e+05,720 days 00:00:00,23.000000
75%,NaN,263376.000000,NaN,290072.250000,NaN,NaN,1.120900e+04,1.141300e+04,2022-10-24 00:00:00,2022-11-01 00:00:00,...,10.000000,24.000000,4.000000,295.000000,NaN,NaN,1.030059,3.500840e+05,740 days 00:00:00,40.000000
max,NaN,312737.000000,NaN,375320.000000,NaN,NaN,2.300000e+07,2.541500e+07,2024-11-14 00:00:00,2025-10-27 00:00:00,...,12.000000,31.000000,6.000000,365.000000,NaN,NaN,3.164865,4.604111e+07,1137 days 00:00:00,252.000000


In [8]:
df[['due_date','total_amount_to_repay','amount_funded_by_lender','lender_portion_funded','lender_portion_to_be_repaid','pct_repay','target']].head(50)

,due_date,total_amount_to_repay,amount_funded_by_lender,lender_portion_funded,lender_portion_to_be_repaid,pct_repay,target
0,2022-09-06,8448.0,120.85,0.014305,121.0,1.000000,0.0
1,2022-08-06,25979.0,7768.50,0.300000,7794.0,1.003244,0.0
2,2024-09-13,7142.0,1380.00,0.200000,1428.0,1.035072,0.0
3,2022-10-27,9233.0,2687.40,0.300000,2770.0,1.030699,0.0
4,2022-12-05,4728.0,1369.20,0.300000,1418.0,1.035933,0.0
5,2022-10-26,4325.0,1297.50,0.300000,1298.0,1.000000,0.0
6,2022-12-06,6735.0,2020.50,0.300000,2021.0,1.000000,0.0
7,2022-11-01,1562.0,462.00,0.300000,469.0,1.014286,0.0
8,2022-10-18,21994.0,0.00,0.000000,0.0,1.023167,0.0
9,2022-08-22,18852.0,5655.60,0.300000,5656.0,1.000000,0.0


In [9]:
date_cols = [col for col in df.columns if 'date' in col]
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

In [10]:
last_loan_day = max(df['disbursement_date'])
recency = df[['customer_id', 'total_amount','disbursement_date']]
recency['recency'] = last_loan_day - recency['disbursement_date']
rfm = recency.groupby('customer_id').agg({'total_amount':'sum','recency':'min'})
rfm = rfm.rename(columns={'total_amount':'monetary'})
frequency = df['customer_id'].value_counts()
frequency.name = 'frequency'
df = df.join(rfm, on='customer_id', how='left')
df = df.join(frequency, on='customer_id', how='left')

In [25]:
rfm

,monetary,recency
customer_id,,
145,14500.0,981 days
161,11025.0,304 days
166,5881.0,979 days
1487,7000.0,976 days
4534,50130.0,490 days
...,...,...
312631,19160.0,1 days
312651,23000.0,1 days
312654,10000.0,10 days


In [29]:
recency_pivot = df[~df['target'].isna()].groupby('recency')[['monetary','frequency','duration','target']].mean()
recency_pivot.tail(50)

# plt.figure(figsize=(15,15))

,monetary,frequency,duration,target
recency,,,,
975 days,40000.000000,1.000000,60.000000,0.000000
976 days,85372.727273,3.000000,15.454545,0.090909
977 days,21473.500000,1.000000,30.000000,0.000000
978 days,1355.000000,1.000000,30.000000,0.000000
979 days,52737.750000,2.500000,37.666667,0.333333
980 days,32934.166667,2.500000,22.000000,0.333333
981 days,29603.600000,1.400000,30.000000,0.400000
982 days,2595.000000,4.333333,30.000000,0.166667
983 days,19481.666667,1.000000,30.000000,0.333333


In [35]:
df[(df['new_versus_repeat']==1)&(df['frequency']>=1)&(df['target']==1)][['customer_id','loan_type','lender_id','duration','lender_portion_funded','monetary','frequency','target']]

,customer_id,loan_type,lender_id,duration,lender_portion_funded,monetary,frequency,target
105,257748,O,lender_C,30,1.0,10000.0,1,1.0
901,266502,B,lender_A,7,0.3,1500.0,1,1.0
1063,308490,H,lender_A,7,0.2,10000.0,2,1.0
1178,250504,O,lender_C,30,1.0,10000.0,1,1.0
1696,270330,B,lender_A,7,0.3,1878.0,1,1.0
...,...,...,...,...,...,...,...,...
66193,310612,H,lender_A,7,0.2,9740.0,2,1.0
67032,257750,O,lender_C,30,1.0,5000.0,1,1.0
67633,250503,O,lender_C,30,1.0,10000.0,1,1.0
67992,263379,B,lender_A,7,0.3,1900.0,1,1.0


In [23]:
recency['recency']

,customer_id,total_amount,disbursement_date,recency
0,266671,8448.00,2022-08-30,807 days
1,248919,25895.00,2022-07-30,838 days
2,308486,6900.00,2024-09-06,69 days
3,266004,8958.00,2022-10-20,756 days
4,253803,4564.00,2022-11-28,717 days
...,...,...,...,...
18589,297596,1730.41,2023-02-09,644 days
18590,259715,1534.00,2022-08-04,833 days
18591,296701,1372.21,2022-06-23,875 days
18592,268271,5608.00,2022-08-23,814 days


In [18]:
df['frequency'].value_counts()

frequency
14    2464
16    2448
19    2394
15    2370
18    2214
      ... 
85      85
81      81
80      80
78      78
67      67
Name: count, Length: 113, dtype: int64